# HW3
This program retrieves eia government API data and writes it to a sqlite3 db.

In [20]:
import config
import sqlite3
import requests
import pandas as pd

In [21]:
# get data
EIA_API_KEY = config.EIA_API_KEY
API_ROUTE = config.EIA_API_ROUTE
data = None
try:
    response = requests.get(API_ROUTE, headers={"X-Api-Key": EIA_API_KEY})
    response.raise_for_status()  # Raise an HTTPError for bad responses
    data = response.json()
    data = data['response']['data']
except requests.exceptions.HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
except Exception as err:
    print(f"Other error occurred: {err}")

In [22]:
# put the data into a pandas dataframe
if data is None:
    print("Data is None")
    exit()
df = pd.DataFrame(data)
# Sort by time period
df_sorted = df.sort_values(by='period', ascending=False)

In [23]:
# write to sqlite
conn = sqlite3.connect('energy_data.db')
cursor = conn.cursor()
# Drop the table if it already exists
cursor.execute('DROP TABLE IF EXISTS energy_data')
# Create the table with adjusted column names
cursor.execute('''CREATE TABLE IF NOT EXISTS energy_data (
                    period TEXT,
                    product_name TEXT,
                    activity_name TEXT,
                    value REAL,
                    unit TEXT)''')

# Insert the data into the table
for _, row in df.iterrows():
    value = row['value']
    
    if value == 'w' or value == '--':
        value = None
    
    cursor.execute('''INSERT INTO energy_data (period, product_name, activity_name, value, unit) 
                      VALUES (?, ?, ?, ?, ?)''', 
                      (row['period'], row['productName'], row['activityName'], value, row['unit']))

conn.commit()
conn.close()


In [25]:
# query the most popular (most consumed by value) energy source per year
conn = sqlite3.connect('energy_data.db')
cursor = conn.cursor()

cursor.execute('''SELECT period, product_name, MAX(value) as max_value
                    FROM energy_data
                    WHERE activity_name = 'Consumption'
                    AND product_name != 'Primary energy'
                    GROUP BY period''')

rows = cursor.fetchall()
for row in rows:
    print(row)
    
conn.close()

('2011', 'Petroleum and other liquids', 37818914.1932261)
('2012', 'Refined petroleum products', 36968829.66348722)
('2013', 'Refined petroleum products', 37665936.11771712)
('2014', 'Refined petroleum products', 37958618.21962755)
('2015', 'Refined petroleum products', 38763024.61687428)
('2016', 'Refined petroleum products', 39241938.14488688)
('2017', 'Refined petroleum products', 39588724.18721048)
('2018', 'Refined petroleum products', 40466262.94988344)
('2019', 'Refined petroleum products', 40439642.89580417)
('2020', 'Refined petroleum products', 35496819.50868142)


In [26]:
# find the average total energy consumption, and production over the total time span
conn = sqlite3.connect('energy_data.db')
cursor = conn.cursor()

cursor.execute('''
    SELECT 'Consumption' as activity, AVG(value) as avg_value
    FROM energy_data
    WHERE activity_name = 'Consumption'
    AND product_name = 'Primary energy'
    UNION
    SELECT 'Production' as activity, AVG(value) as avg_value
    FROM energy_data
    WHERE activity_name = 'Production'
    AND product_name = 'Primary energy'
''')

rows = cursor.fetchall()
for row in rows:
    print(row)
    
conn.close()


('Consumption', 33079310.7404043)
('Production', 29814652.43431527)


In [27]:
# transaction to remove all export data, and then rollback
conn = sqlite3.connect('energy_data.db')
cursor = conn.cursor()

cursor.execute('''
    SELECT activity_name, COUNT(*)
    FROM energy_data
    GROUP BY activity_name
''')

rows = cursor.fetchall()
print("Initial rows:", rows)

cursor.execute('BEGIN TRANSACTION')
cursor.execute('''
    DELETE FROM energy_data
    WHERE activity_name = 'Export'
    SELECT activity_name, COUNT(*)
    FROM energy_data
    GROUP BY activity_name
''')

rows = cursor.fetchall()
print("Rows after delete:", rows)
    
cursor.execute('ROLLBACK')

cursor.execute('''
    SELECT activity_name, COUNT(*)
    FROM energy_data
    GROUP BY activity_name
''')

rows = cursor.fetchall()
print("Rows after rollback:", rows)

conn.close()

Initial rows: [('Bunker', 12), ('Capacity', 130), ('Consumption', 1112), ('Distribution losses', 10), ('Emissions', 70), ('Exports', 462), ('GDP', 20), ('Generation', 470), ('Imports', 462), ('Net imports', 40), ('Population', 20), ('Production', 810), ('Reserves', 10), ('Stocks, OECD', 10)]


OperationalError: near "DELETE": syntax error